In [ ]:
import json
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# NOTE: Use python version >= 3.10 since I use match statements

# Optimization Final Project
**Joseph Andrew Sharp Luevano**

## Introduction

One of my favourite video games is Slay the Spire, a card-based adventure game where players progress through different levels, battling enemies and collecting cards to create a powerful deck. In each game, levels and challenges are randomly generated. If your character dies at any point, you must start a new game from the beginning, losing all items and progress from that run. It is a challenging, yet rewarding game where you must manage risk, plan carefully, and reason about probabilities, all while building a sufficiently powerful deck to defeat the final boss.

The game is divided into 3 acts, each with their own [map](https://steamuserimages-a.akamaihd.net/ugc/957481092845833078/A708E7902EE6E6EFECD06D1FB5FB906BB9EB8D51/). Each node in the map represents a type of encounter, such as a fight or a shop. Between each encounter, the player chooses the next node they wish to visit in the map. Choosing which path to take is an important and complicated process, as it involves many subtle variables. In general, the player should choose the path that they believe will set them up for the best chance of winning the run while not dying. The player's decision can be boiled down to choosing the path that contains the nodes they believe will help them achieve his end. Each type of node provides a different set of risks and rewards, so the player must decide how much they value a node's rewards and whether it outweighs the risk. Evaluating a particular node is a difficult process, but a player must estimate their values if they wish to reason about which path to take. The goal of my algorithm is to find the path in the map that maximizes utility given some evaluation of the nodes.

I am aware that such this algorithm will necessarily be unable to capture every subtlety of the decision process, but will nonetheless be a helpful tool in analyzing potential routes. This type of map navigation is not unique to Slay the Spire, as it is common among other games in the roguelike genre.  Therefore, this algorithm will also outline a general process that can be applied to other games with this feature.

## The Problem

The problem of finding the best path in Slay the Spire can be represented by the *Longest Dipath Problem*. The problem is as follows: Given a directed graph, we wish to find the longest weighted path from some start node to an end node. The 'length' of a path is determined by the total summed weight of the edges taken along the path.

Alan Sultan describes a way to formulate this as a linear program in section 1.13 of *"Linear Programming. An Introduction with Applications"*, which we can then solve for the longest path:

Let $D = (E, V)$ be a directed graph with vertices $V = \{1, ..., n\}$, where $1$ is the starting node and $n$ is the ending node. Let $f : V \rightarrow \mathbb{R}$ be a function that takes a node and returns its value (or weight). We define the variables $x_{ij} \in \{0, 1\}$ for each edge in $E$ starting at vertex $i$ and ending at $j$. In our linear program, $x_{ij} = 1$ if the edge $(i, j)$ is included in the path, and $0$ otherwise. Therefore, the weight of the path is given by the sum of the edges chosen multiplied by their respective weights. Note that in our case, the weight of an edge is equivalent to the weight of the vertex the edge arrives at. This gives us our objective function we wish to maximize:
$$c = \sum f(j)\cdot x_{ij}$$

The constraints are determined by the vertices, and enforce that the proper number of edges are entering and leaving the vertex. Therefore, we will have $|V|$ constraints. The first constraint is that the number of edges leaving the starting vertex ($1$) is exactly $1$, giving us:
$$\sum x_{1j} = 1$$

Next, we add a constraint ensuring that exactly one edge arrives at the end vertex ($n$):
$$\sum x_{in} = 1$$

Finally, we must ensure that the number of edges entering a vertex equals the number of edges leaving it:
$$\sum x_{ij} = \sum x_{jk}$$

We also add the constraint that all the variables are non-negative and integral. The author claims that once an optimal solution is obtained for this LP, the variables which are $1$ determine which edges will be included in the path. This is the method I will be using to solve the problem.

In my algorithm, the graph will be represented with an incidence matrix, $M \in \mathbb{R}^{|V| \times |E|}$. Each row represents a vertex, and each column represents and edge. The entries are defined as follows:
$$M_{uv} = \begin{cases}
1 & \text{If edge $v$ arrives at vertex $u$} \\
-1 & \text{ If edge $v$ leaves vertex $u$} \\
0 & \text{otherwise}
\end{cases}$$

Consider the constraint, $\sum x_{ij} = \sum x_{jk}$. It can be rewritten as $\sum x_{ij} - \sum x_{jk} = 0$. Note that the edges arriving at $j$ have a coefficient of $1$, and the edges leaving $j$ have a coefficient of $-1$. Furthermore, we can write it as $\sum x_{ij} - \sum x_{jk} + 0 \cdot \sum x_{ab} = 0$ where $x_{ab}$ are the remaining edges. Observe that the coefficients have the same meaning as in our incidence matrix. We can rearrange the order of summation so that the coefficients are equal to the entries in the corresponding row of $M$. We can do this to every constraint so that the left-hand side coefficients are given by the incidence matrix. Note that the final constraint does not need to be changed, since there are only edges arriving at the last vertex (with entries $1$). We can multiply the first constraint by $-1$ so that the coefficients on the left-hand side are negative, like in the incidence matrix, giving us $\sum -x_{1j} = -1$. This allows us to simplify the linear program to the following:

$$\text{maximize: } c = \sum f(j)\cdot x_{ij}$$
$$\text{subject to: } Mx = [-1, 0, ..., 0, 1]^T$$
$$x \geq 0$$

# My Solution

## Initializing the Map
The maps in Slay the Spire are randomly generated from a seed. I will be using [this program](https://github.com/Ru5ty0ne/sts_map_oracle) by a Reddit user who [reverse engineered](https://www.reddit.com/r/slaythespire/comments/ndqweh/i_have_reverseengineered_map_generation_algorithm/) the game's map making algorithm. The program takes a seed and creates a json file containing the map for all three acts. The json object has the following structure:
```
{
"edges": [{"src_x": _, "src_y": _, "dst_x": _, "dst_y": _}],
"nodes": [{"x": _, "y": _, "class": _}]
}
```
The $x$ coordinate may range from $0$ to $6$, while the $y$ coordinate ranges from $0$ to $14$. The "class" attribute tells us what type of node it is. Here is a short summary of the types of nodes.

* "MonsterRoom": A standard fight.
* "ShopRoom": A place to purchase items for gold.
* "RestRoom": A place to upgrade cards or restore health.
* "MonsterRoomElite": A difficult fight with greater rewards.
* "EventRoom": A random event, which has a varied outcome.
* "TreasureRoom": A chest containing a reward.

I have included some pre-generated map json files. The first step is to import the json data.

In [2]:
json_file_path = './maps/673465884448_Act1.json'
with open(json_file_path, 'r') as f:
    json_data = json.load(f)

json_data

{'edges': [{'src_x': 0, 'src_y': 0, 'dst_x': 1, 'dst_y': 1},
  {'src_x': 2, 'src_y': 0, 'dst_x': 3, 'dst_y': 1},
  {'src_x': 5, 'src_y': 0, 'dst_x': 4, 'dst_y': 1},
  {'src_x': 1, 'src_y': 1, 'dst_x': 0, 'dst_y': 2},
  {'src_x': 3, 'src_y': 1, 'dst_x': 2, 'dst_y': 2},
  {'src_x': 3, 'src_y': 1, 'dst_x': 3, 'dst_y': 2},
  {'src_x': 3, 'src_y': 1, 'dst_x': 4, 'dst_y': 2},
  {'src_x': 4, 'src_y': 1, 'dst_x': 5, 'dst_y': 2},
  {'src_x': 0, 'src_y': 2, 'dst_x': 1, 'dst_y': 3},
  {'src_x': 2, 'src_y': 2, 'dst_x': 3, 'dst_y': 3},
  {'src_x': 3, 'src_y': 2, 'dst_x': 4, 'dst_y': 3},
  {'src_x': 4, 'src_y': 2, 'dst_x': 5, 'dst_y': 3},
  {'src_x': 5, 'src_y': 2, 'dst_x': 6, 'dst_y': 3},
  {'src_x': 1, 'src_y': 3, 'dst_x': 1, 'dst_y': 4},
  {'src_x': 3, 'src_y': 3, 'dst_x': 3, 'dst_y': 4},
  {'src_x': 4, 'src_y': 3, 'dst_x': 3, 'dst_y': 4},
  {'src_x': 4, 'src_y': 3, 'dst_x': 5, 'dst_y': 4},
  {'src_x': 5, 'src_y': 3, 'dst_x': 6, 'dst_y': 4},
  {'src_x': 6, 'src_y': 3, 'dst_x': 6, 'dst_y': 4},
  {

We will extend the node list and edge list to include a source node and sink node. The source will be connected to every node in the first level (y coordinate 0), and the sink will be connected to every node in the last level (y coordinate 14).

In [3]:
node_list = json_data['nodes'].copy()

# Source is represented by coordinates (-1, -1)
node_list.append({"x": -1, "y": -1, "class": "Source"})

# Sink is represented by coordinates (inf, inf)
node_list.append({"x": np.inf, "y": np.inf, "class": "Sink"})

# Sort nodes in order of increasing y coord., then increasing x coord.
node_list.sort(key=lambda n: (n['y'], n['x']))

edge_list = json_data['edges'].copy()
# Add edges from source to first level
edge_list = [{"src_x": -1, "src_y": -1, "dst_x": node['x'], "dst_y": node['y']}
             for node in node_list
             if node['y'] == 0] + edge_list

# Add edges from last level to destination
edge_list = edge_list + [{"src_x": node['x'], "src_y": node['y'], "dst_x": np.inf, "dst_y": np.inf}
                         for node in node_list
                         if node['y'] == 14]

edge_list.sort(key=lambda e: (e['dst_y'], e['dst_x']))

Next, we will use this information to construct an incidence matrix. Note that the constraints could be built without the incidence matrix, but storing the graph this way is common and allows you to do other things with the structure.

In [4]:
num_nodes = len(node_list)
num_edges = len(edge_list)
incidence_matrix = np.zeros((num_nodes, num_edges), dtype=int)

# Keeps track of matrix row corresponding to a node
node_index = {(node['x'], node['y']): index for index, node in enumerate(node_list)}

# Update the incidence matrix
for col, edge in enumerate(edge_list):
    src = (edge['src_x'], edge['src_y'])
    dst = (edge['dst_x'], edge['dst_y'])

    src_index = node_index[src]
    dst_index = node_index[dst]

    incidence_matrix[src_index, col] = -1
    incidence_matrix[dst_index, col] = 1

print(incidence_matrix)

[[-1 -1 -1 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0 -1]
 [ 0  0  0 ...  1  1  1]]


## Evaluating Nodes

Next, we will create a list of the weights corresponding to each edge. Recall that the weight is given by the value of the node it arrives at. To implement this we must create a function that evaluates a node. This is where the process can get extremely difficult, as the exact value of a node relies on many variables, some of which we have no information about without knowing the state of the game. To resolve this, we will use a simple evaluation function. Note that the function can get as complex as you want by considering more variables, and could perhaps be trained by machine learning.

In [ ]:
# Ranked nodes
def rank_eval(node_type):
    match node_type:
        case "MonsterRoom":
            return 3
        case "ShopRoom":
            return 1
        case "RestRoom":
            return 4
        case "MonsterRoomElite":
            return 5
        case "EventRoom":
            return 2
        case "TreasureRoom":
            return 0 # Every path contains exactly 1 treasure room
        case "Source" | "Sink":
            return 0

# Score penalized by potential risk, multiplied by risk tolerance 0 <= tol <= 1
# evaluation = value - risk * (1 - tol)
def risk_eval(node_type, tol=1.0):
    match node_type:
        case "MonsterRoom":
            return 30 - 15*(1 - tol)
        case "ShopRoom":
            return 10 - 0*(1 - tol)
        case "RestRoom":
            return 40 - 0*(1 - tol)
        case "MonsterRoomElite":
            return 50 - 50*(1 - tol)
        case "EventRoom":
            return 20 - 5*(1 - tol)
        case "TreasureRoom":
            return 0 # Every path contains exactly 1 treasure room
        case "Source" | "Sink":
            return 0

"""
# Some complicated evaluation function
# Each node's value can depend on a large number of variables
def complicated_eval(node_type, data):
    match node_type:
        case "MonsterRoom":
            return 10*(data['potion_chance']) \
                + 20*(data['prob_useful_card']) \
                - 20*(data['expected_damage_taken']) \
                ...


"""

# Choose the evaluation function to use
# Feel free to change and experiment with it
eval_fn = lambda node_type: risk_eval(node_type, 1)

These evaluation functions can be tweaked to find a good score for each type of node.

Next, we will create a list of weights for each edge, according to the value of the node it arrives at.

In [ ]:
node_types = {(node['x'], node['y']): node['class'] for node in node_list}
edge_weights = [eval_fn(node_types[(edge['dst_x'], edge['dst_y'])])
                for edge in edge_list]

## Constructing the LP

Now that we have our incidence matrix and weighed the edges, we can construct and solve the linear program. I will be using the ```linprog``` function from scipy. By default, the function minimizes the objective function, so we must multiply the objective function my -1 to get a maximization problem.

In [ ]:
# Multiply objective function coefficients by -1
edge_weights = list(map(lambda x: -1*x, edge_weights))

# Recall that the incidence matrix gives us the LHS of the constraints
# Create LHS of constraints
rhs = [0 for _ in node_list]
rhs[0] = -1
rhs[-1] = 1

# Create variable bounds
bounds = [(0, np.inf) for _ in edge_list]

# Use linprog to find optimal path
opt = linprog(c=edge_weights, A_eq=incidence_matrix, b_eq=rhs, bounds=bounds, integrality=1)
print(opt)

If the optimization is successful, then we have found this longest path we desired. The value 'fun' gives us the total weight of the edges. The 'x' value tells us which edges were taken. We will use this 'x' value to see which nodes were selected.

In [ ]:
# Find chosen edges
chosen_edges = []
for i in range(len(opt.x)):
    if opt.x[i] == 1:
        chosen_edges.append(edge_list[i])
chosen_edges

We will display the map on a grid and highlight the chosen edges.

In [ ]:
# Code for generating graph
def create_grid(legend):
    # Create an empty 7x15 grid of strings
    grid = np.empty((7, 15), dtype=object)

    # Fill the grid with an empty string
    grid[:] = ""

    # Iterate through the legend and replace the corresponding grid values
    for key, value in legend.items():
        grid[key[0], key[1]] = value

    return grid

def plot_grid(grid, colors):
    grid = np.transpose(grid)  # Transpose the grid

    fig, ax = plt.subplots()

    for i in range(15):
        for j in range(7):
            square_color = colors[grid[i, j]]
            square = patches.Rectangle((j, i), 1, 1, facecolor=square_color, edgecolor="black")
            ax.add_patch(square)

    ax.set_xlim(0, 7)
    ax.set_ylim(0, 15)
    ax.set_aspect("equal")
    ax.axis("off")

    plt.show()

In [ ]:
# Generating graph
# Label each type of node on the 7x15 grid
legend = {(i, j): node_types.get((i, j), "Nothing") for i in range(7) for j in range(15)}

# Label the nodes chosen by the LP
# Set this to False to see original map
if True:
    for edge in chosen_edges[:-1]:
        n = (edge['dst_x'], edge['dst_y'])
        legend[n] = 'Chosen'

# Color mapping (Can be changed if you like)
color_map = {
        'Nothing': "black",
        'MonsterRoom': "red",
        'MonsterRoomElite': "darkred",
        'ShopRoom': "orange",
        'RestRoom': "purple",
        'TreasureRoom': "green",
        'EventRoom': "yellow",
        'Chosen': "white"
    }

grid = create_grid(legend)
plot_grid(grid, color_map)
print([f"{(e['dst_x'], e['dst_y'])} - {node_types[(e['dst_x'], e['dst_y'])]}" for e in chosen_edges])

## Results

